In [10]:

from multiprocessing import Pool, cpu_count
import numpy as np
from src.input.write_files import write_circuit_file
from src.input.read_files import find_files, numbers_from_file_name, normal_sec_size_int_from_file_name
from datetime import datetime
from src.algorithm.circuit_synthesis_vanilla import synthesise_circuit
from src.algorithm.execution import load_circuit_and_benchmark
import csv

print(f'There are {cpu_count()} threads available on this machine.')

There are 12 threads available on this machine.


In [32]:
sample_size = 20
lower_size = 10
upper_size = 100
step_size = 1

sizes = np.arange(lower_size, upper_size + step_size, step_size)
samples = np.arange(1, sample_size + 1, 1)

input_gen_processors = 10

input_dir = r'resources\runtime_performance\input_circuits'
input_file_prefix = 'Circuit'

benchmark_processors = 10

output_dir = r'resources\runtime_performance'
output_file_prefix = 'RuntimePerformance'

output_file_fieldnames = ['size', 'run', 'sec_size', 'num_gates', 'process_time', 'initial_rss', 'final_rss']

In [25]:
input_gen_argument_tuples = ([(size, size * size, rf'..\{input_dir}\{input_file_prefix}_{size}_{sample}.txt') for size in sizes for sample in samples])

[print(argument_tuple) for argument_tuple in input_gen_argument_tuples[:min(10, len(input_gen_argument_tuples))]]

(10, 100, '..\\resources\\runtime_performance\\input_circuits\\Circuit_10_1.txt')
(10, 100, '..\\resources\\runtime_performance\\input_circuits\\Circuit_10_2.txt')
(10, 100, '..\\resources\\runtime_performance\\input_circuits\\Circuit_10_3.txt')
(10, 100, '..\\resources\\runtime_performance\\input_circuits\\Circuit_10_4.txt')
(10, 100, '..\\resources\\runtime_performance\\input_circuits\\Circuit_10_5.txt')
(10, 100, '..\\resources\\runtime_performance\\input_circuits\\Circuit_10_6.txt')
(10, 100, '..\\resources\\runtime_performance\\input_circuits\\Circuit_10_7.txt')
(10, 100, '..\\resources\\runtime_performance\\input_circuits\\Circuit_10_8.txt')
(10, 100, '..\\resources\\runtime_performance\\input_circuits\\Circuit_10_9.txt')
(10, 100, '..\\resources\\runtime_performance\\input_circuits\\Circuit_10_10.txt')


[None, None, None, None, None, None, None, None, None, None]

In [26]:
print(f'Generating {len(input_gen_argument_tuples)} circuits in size range from {lower_size} to {upper_size} ...')

if __name__ ==  '__main__':
    with Pool(processes = input_gen_processors) as pool:
        pool.starmap(write_circuit_file, input_gen_argument_tuples)

print(f'Saved circuits to {input_dir} ...')

Generating 1820 circuits in size range from 10 to 100 ...
Saved circuits to resources\runtime_performance\input_circuits ...


In [27]:
file_regex = rf'({input_file_prefix}).*\.txt'
size_regex = r'_\d+_'
sample_regex = r'_\d+\.'

input_file_names = find_files(rf'..\{input_dir}', file_regex)
input_file_names.sort(key = lambda input_file_name: numbers_from_file_name(input_file_name, size_regex, sample_regex))

[print(input_file_name) for input_file_name in input_file_names[:min(10, len(input_file_names))]]

Circuit_10_1.txt
Circuit_10_2.txt
Circuit_10_3.txt
Circuit_10_4.txt
Circuit_10_5.txt
Circuit_10_6.txt
Circuit_10_7.txt
Circuit_10_8.txt
Circuit_10_9.txt
Circuit_10_10.txt


[None, None, None, None, None, None, None, None, None, None]

In [28]:
file_name_arguments = [(input_file_name, normal_sec_size_int_from_file_name(input_file_name, size_regex)) for input_file_name in input_file_names]
benchmark_argument_tuples = [(synthesise_circuit, input_dir, input_file_name, sec_size) for (input_file_name, sec_size) in file_name_arguments]

[print(argument_tuple) for argument_tuple in benchmark_argument_tuples[:min(10, len(benchmark_argument_tuples))]]

(<function synthesise_circuit at 0x0000025B464E1C10>, 'resources\\runtime_performance\\input_circuits', 'Circuit_10_1.txt', 2)
(<function synthesise_circuit at 0x0000025B464E1C10>, 'resources\\runtime_performance\\input_circuits', 'Circuit_10_2.txt', 2)
(<function synthesise_circuit at 0x0000025B464E1C10>, 'resources\\runtime_performance\\input_circuits', 'Circuit_10_3.txt', 2)
(<function synthesise_circuit at 0x0000025B464E1C10>, 'resources\\runtime_performance\\input_circuits', 'Circuit_10_4.txt', 2)
(<function synthesise_circuit at 0x0000025B464E1C10>, 'resources\\runtime_performance\\input_circuits', 'Circuit_10_5.txt', 2)
(<function synthesise_circuit at 0x0000025B464E1C10>, 'resources\\runtime_performance\\input_circuits', 'Circuit_10_6.txt', 2)
(<function synthesise_circuit at 0x0000025B464E1C10>, 'resources\\runtime_performance\\input_circuits', 'Circuit_10_7.txt', 2)
(<function synthesise_circuit at 0x0000025B464E1C10>, 'resources\\runtime_performance\\input_circuits', 'Circui

[None, None, None, None, None, None, None, None, None, None]

In [29]:
output_file_timestamp = datetime.now().strftime("%d%m%y_%H%M")

print(f'Benchmarking runtime performance in size range from {lower_size} to {upper_size} in {benchmark_processors} threads ...')

if __name__ ==  '__main__':
    with Pool(processes = benchmark_processors) as pool:
        result_tuples = pool.starmap(load_circuit_and_benchmark, benchmark_argument_tuples)

print(f"Saving results to '{output_dir} ...")

with open(rf'..\{output_dir}\{output_file_prefix}_{output_file_timestamp}.csv', mode='w', newline='') as output_file:
    writer = csv.DictWriter(output_file, fieldnames=output_file_fieldnames)
    writer.writeheader()

    for (size, run, sec_size, num_gates, process_time, initial_rss, final_rss) in result_tuples:

        writer.writerow({'size': size, 'run': run, 'sec_size': sec_size, 'num_gates': num_gates, 'process_time': process_time, 'initial_rss': initial_rss, 'final_rss': final_rss})

print('Done.')

Benchmarking runtime performance in size range from 10 to 100 in 10 threads ...
Saving results to 'resources\runtime_performance ...
Done.


In [30]:
import pandas as pd

df = pd.read_csv(rf'..\{output_dir}\{output_file_prefix}_{output_file_timestamp}.csv')

#print(df)

df_num_gates_agg = df.groupby(['size', 'sec_size']).agg(num_gates_mean=('num_gates', 'mean'), num_gates_std=('num_gates', 'std'))

#df_agg = df.groupby(['size', 'sec_size']).agg(process_time_mean=('process_time', 'mean'), process_time_std=('process_time', 'std'),
 #                                             num_gates_mean=('num_gates', 'mean'), num_gates_std=('num_gates', 'std'),
   #                                               initial_rss_mean=('initial_rss', 'mean'), initial_rss_std=('initial_rss', 'std'),
     #                                             final_rss_mean=('final_rss', 'mean'), final_rss_std=('final_rss', 'std'))



df_num_gates_agg.head()

sizes = [size for (size, _) in df_num_gates_agg.index.tolist()]

sec_sizes = 500*np.array([sec_size for (_, sec_size) in df_num_gates_agg.index.tolist()])

print(sec_sizes)

[1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
 1000 1000 1000 1000 1000 1000 1000 1000 1000 1500 1500 1500 1500 1500
 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500
 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500
 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500
 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500
 1500 1500 1500 1500 1500 1500 1500]


In [31]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
        x=sizes,
        y=df_num_gates_agg['num_gates_mean'].tolist(),
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=df_num_gates_agg['num_gates_std'].tolist(),
            visible=True)
    ))

fig.add_trace(go.Scatter(
        x=sizes,
        y=sec_sizes))

fig.show()